In [2]:
from influxdb import InfluxDBClient
import pandas as pd
import os
import datetime
import re
import time
import numpy as np
from datetime import timedelta
import joblib

資料前處理

In [3]:
def power_preprocessing(data):
    #將欄位資料從['UpdateTime','kWh']擴充到['UpdateTime','kWh','Hour','Date','power']
    data['UpdateTime'] = pd.to_datetime(data['UpdateTime']) 
    data['Hour'] = data['UpdateTime'].dt.hour
    data['minute'] = data['UpdateTime'].dt.minute
    data['Date'] = data['UpdateTime'].dt.date
    #依據UpdateTime做排序，
    data = data.sort_values(by='UpdateTime')
    #將小時內重複出現的資料刪除，只保留第一筆，並透過get_hour_power去取得原本用電量計算的區間
    get_hour_power = data
    get_hour_power = get_hour_power.drop_duplicates(subset=['Date','Hour'], keep="first")
    return get_hour_power
#     data1['power'] = data1['kWh'].shift(-1) - data1['kWh']

#     return data1

In [4]:
def group(x):
    row = x.copy()
    

In [5]:
def power_preprocessing(data):
    data = data[pd.to_datetime(data['UpdateTime']) >= pd.to_datetime('2022-01-04')]
    data = data[pd.to_datetime(data['UpdateTime']) <= pd.to_datetime('2022-06-08')]
    #將欄位資料從['UpdateTime','kWh']擴充到['UpdateTime','kWh','Hour','Date','power']
    data['UpdateTime'] = pd.to_datetime(data['UpdateTime']) 
    data['minute'] = data['UpdateTime'].dt.minute
    data['Hour'] = data['UpdateTime'].dt.hour
    data['Date'] = data['UpdateTime'].dt.date
    #依據UpdateTime做排序，
    data = data.sort_values(by='UpdateTime')
    #將小時內重複出現的資料刪除，只保留第一筆15001
#     data1 = data.drop_duplicates(subset=['Date','Hour','minute'], keep="first").reset_index(drop=True)
    #將每小時用電量時間分成4塊(0~15、16~30、31~45、46~60)
    data['min_interval'] = data['minute'].map(lambda x: 1 if x<=15 else (2 if x>15 and x<=30 else (3 if x>30 and x<=45 else(4 if x>45 else 0))))
    #     同區間只保留第一筆跟最後一筆
    first = data.drop_duplicates(subset=['Date','Hour','min_interval'], keep="first").reset_index(drop=True)
    mask = (first['min_interval'] == 1)
    first = first[mask]
    last = data.drop_duplicates(subset=['Date','Hour','min_interval'], keep="last").reset_index(drop=True)
    last2 = data.drop_duplicates(subset=['Date','Hour','min_interval'], keep="last").reset_index(drop=True)
    last3 = last2[last2['min_interval'] == 4]
    last2 = last2.drop(last3.index)
    print(last2,'1231321')
#     last2.drop(last2[ last2['minute']== 59 ].index, inplace=True) #在原数据上进行删除
#     last2.drop(last2[ last2['minute']== 58 ].index, inplace=True) #在原数据上进行删除
    # print(last2.head(10))
    answer = pd.concat([first,last,last2],axis=0)
    answer = answer.sort_values(by=['UpdateTime']).reset_index(drop=True)
    answer.to_csv(f'./Dataset/answer.csv')
    answer['group'] = answer.apply(lambda x:group(x))
    print(answer.head(30),'----------------------')
    column= ['Date','Hour','minute']
    answer = answer.sort_values(by=column)
#     根據Hour、Date判斷是否該日期下該小時有八筆資料
    store_date=[]
    store_hour=[]
    for i in tqdm(range(len(answer[['Hour','Date']].value_counts()))):
        if(answer[['Hour','Date']].value_counts().iloc[i]!=8):
            store_date.append(answer[['Hour','Date']].value_counts()[i:].rename_axis(['Hour','Date']).reset_index(name='counts')['Date'])
            store_hour.append(answer[['Hour','Date']].value_counts()[i:].rename_axis(['Hour','Date']).reset_index(name='counts')['Hour'])
#     根據找到在該日期該小時下沒有八筆的資料
    for i in range(len(store_hour)):
        answer.drop(answer.loc[(answer['Date']==store_date[i].values[0]) & (answer['Hour']==store_hour[i].values[0])].index, inplace=True)
#     刪除後重新排序
    answer.sort_values(by=['UpdateTime'],inplace=True)
    answer.reset_index(drop=True,inplace=True)
#     給予新的欄位必須是浮點數才能先減，因為原先資料本身就是浮點數
    answer['Power']=0.0
#     用一個新的dataframe來存最終的資料
    final_data = pd.DataFrame()
    for i in tqdm(range(0,len(answer),2)):
        answer['Power'][i+1]=answer['kWh'][i+1]-answer['kWh'][i]
        final_data=final_data.append(answer[i+1:i+2],ignore_index=True)

    return final_data
#     return answer

資料彙整

In [6]:
from tqdm import tqdm
def data_merge():
    files = ['building', 'dormitory']
    # files = ['dormitory']
    # 抓取資料夾裡面的資料，進行合併
    for file in files:
        for filename in os.listdir(f"./Dataset/NCUE/history/{file}/"):
            if filename=='merge' or filename=='.ipynb_checkpoints':
                continue
            merge_data = pd.DataFrame()
            for name in os.listdir(f"./Dataset/NCUE/history/{file}/{filename}"):
                if filename=='merge' or filename=='.ipynb_checkpoints':
                    continue
                print(name)
                if name=='.ipynb_checkpoints':
                    continue
                data = pd.read_csv(f'./Dataset/NCUE/history/{file}/{filename}/{name}')
                if 'kWh+' in data.columns:
                    data = data.rename(columns={'kWh+':'kWh'})
                data = data[['UpdateTime', 'kWh']]
                merge_data = pd.concat([merge_data,data], axis=0, ignore_index=True)
            #丟入前處理，並將每棟資料重新存取
            merge_data = merge_data.sort_values(by='UpdateTime')
            merge_data = power_preprocessing(merge_data)
            merge_data = merge_data.rename(columns={'UpdateTime':'TIME_TO_INTERVAL'})
            merge_data = merge_data.reset_index(drop=True)
            merge_data.to_csv(f'./Dataset/NCUE/history/temp/{filename}.csv', index=False)
            
    redatadic = {}
    for filename in os.listdir(f"./Dataset/NCUE/history/temp"):
        if (filename[-3:]!='csv'):
            continue
        r = "[A-Za-z0-9#&-]"
    #     print(re.sub(r, '', filename[:-4]), filename[:-4])
        #re.sub:用於將匹配的字串取代成指定字串
        #從新命名
        redataname = re.sub(r, '', filename[:-4])
        print(filename)

        data = pd.read_csv(f'./Dataset/NCUE/history/temp/{filename}')
    #     data_nan = grab_breakpoint(data)
    #     data = remove_interruption(data_nan,data)

        #將同名字的館院資料合併
        if redataname not in redatadic.keys():
            redatadic[redataname] = data.copy()
        else:
            redatadic[redataname] = pd.concat([redatadic[redataname], data],axis=0, ignore_index=True)
    #             print(redatadic[redataname])


    # # redatadics = redatadic.copy()
    #
    for d in redatadic.keys():
        print(d)
        #將不同館但同時段的資料合併加總，index重算
        redatadic[d] = redatadic[d].groupby(['Date', 'Hour','minute','min_interval']).sum().reset_index()
        redatadic[d]['TIME_TO_INTERVAL'] = redatadic[d].apply(lambda x: "{} {:02d}:{:02d}:00".format(x['Date'],x['Hour'],x['minute']), axis=1)

    for i in redatadic:
        redatadic[i].to_csv(f'./Dataset/NCUE/history/merge/{i}.csv', index=False)

In [ ]:
data_merge()

第九宿舍1L1A_第九宿舍1L1A_0.csv
第九宿舍1L1A_第九宿舍1L1A_10000.csv
第九宿舍1L1A_第九宿舍1L1A_100000.csv
第九宿舍1L1A_第九宿舍1L1A_105000.csv
第九宿舍1L1A_第九宿舍1L1A_110000.csv
第九宿舍1L1A_第九宿舍1L1A_115000.csv
第九宿舍1L1A_第九宿舍1L1A_120000.csv
第九宿舍1L1A_第九宿舍1L1A_125000.csv
第九宿舍1L1A_第九宿舍1L1A_130000.csv
第九宿舍1L1A_第九宿舍1L1A_135000.csv
第九宿舍1L1A_第九宿舍1L1A_140000.csv
第九宿舍1L1A_第九宿舍1L1A_145000.csv
第九宿舍1L1A_第九宿舍1L1A_15000.csv
第九宿舍1L1A_第九宿舍1L1A_150000.csv
第九宿舍1L1A_第九宿舍1L1A_155000.csv
第九宿舍1L1A_第九宿舍1L1A_160000.csv
第九宿舍1L1A_第九宿舍1L1A_165000.csv
第九宿舍1L1A_第九宿舍1L1A_170000.csv
第九宿舍1L1A_第九宿舍1L1A_175000.csv
第九宿舍1L1A_第九宿舍1L1A_180000.csv
第九宿舍1L1A_第九宿舍1L1A_185000.csv
第九宿舍1L1A_第九宿舍1L1A_190000.csv
第九宿舍1L1A_第九宿舍1L1A_195000.csv
第九宿舍1L1A_第九宿舍1L1A_20000.csv
第九宿舍1L1A_第九宿舍1L1A_200000.csv
第九宿舍1L1A_第九宿舍1L1A_205000.csv
第九宿舍1L1A_第九宿舍1L1A_210000.csv
第九宿舍1L1A_第九宿舍1L1A_215000.csv
第九宿舍1L1A_第九宿舍1L1A_220000.csv
第九宿舍1L1A_第九宿舍1L1A_25000.csv
第九宿舍1L1A_第九宿舍1L1A_30000.csv
第九宿舍1L1A_第九宿舍1L1A_35000.csv
第九宿舍1L1A_第九宿舍1L1A_40000.csv
第九宿舍1L1A_第九宿舍1L1A_45000.csv
第九宿舍1L1A_第九宿舍1L1A_5000.csv


  0%|                                                                                        | 0/14232 [00:00<?, ?it/s]C:\Users\idsl\anaconda3\envs\sister\lib\site-packages\ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|███████████████████████████████████████████████████████████████████████████| 14232/14232 [00:47<00:00, 300.37it/s]


第九宿舍1L1B_第九宿舍1L1B_0.csv
第九宿舍1L1B_第九宿舍1L1B_10000.csv
第九宿舍1L1B_第九宿舍1L1B_100000.csv
第九宿舍1L1B_第九宿舍1L1B_105000.csv
第九宿舍1L1B_第九宿舍1L1B_110000.csv
第九宿舍1L1B_第九宿舍1L1B_115000.csv
第九宿舍1L1B_第九宿舍1L1B_120000.csv
第九宿舍1L1B_第九宿舍1L1B_125000.csv
第九宿舍1L1B_第九宿舍1L1B_130000.csv
第九宿舍1L1B_第九宿舍1L1B_135000.csv
第九宿舍1L1B_第九宿舍1L1B_140000.csv
第九宿舍1L1B_第九宿舍1L1B_145000.csv
第九宿舍1L1B_第九宿舍1L1B_15000.csv
第九宿舍1L1B_第九宿舍1L1B_150000.csv
第九宿舍1L1B_第九宿舍1L1B_155000.csv
第九宿舍1L1B_第九宿舍1L1B_160000.csv
第九宿舍1L1B_第九宿舍1L1B_165000.csv
第九宿舍1L1B_第九宿舍1L1B_170000.csv
第九宿舍1L1B_第九宿舍1L1B_175000.csv
第九宿舍1L1B_第九宿舍1L1B_180000.csv
第九宿舍1L1B_第九宿舍1L1B_185000.csv
第九宿舍1L1B_第九宿舍1L1B_190000.csv
第九宿舍1L1B_第九宿舍1L1B_195000.csv
第九宿舍1L1B_第九宿舍1L1B_20000.csv
第九宿舍1L1B_第九宿舍1L1B_200000.csv
第九宿舍1L1B_第九宿舍1L1B_205000.csv
第九宿舍1L1B_第九宿舍1L1B_210000.csv
第九宿舍1L1B_第九宿舍1L1B_215000.csv
第九宿舍1L1B_第九宿舍1L1B_220000.csv
第九宿舍1L1B_第九宿舍1L1B_225000.csv
第九宿舍1L1B_第九宿舍1L1B_230000.csv
第九宿舍1L1B_第九宿舍1L1B_25000.csv
第九宿舍1L1B_第九宿舍1L1B_30000.csv
第九宿舍1L1B_第九宿舍1L1B_35000.csv
第九宿舍1L1B_第九宿舍1L1B_40000.c

  0%|                                                                                        | 0/14232 [00:00<?, ?it/s]C:\Users\idsl\anaconda3\envs\sister\lib\site-packages\ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 31%|███████████████████████▎                                                    | 4357/14232 [00:12<00:30, 320.69it/s]

In [1]:
answer = pd.read_csv(f'./Dataset/answer.csv')
answer

NameError: name 'pd' is not defined

In [230]:
from tqdm import tqdm

In [231]:
answer[['Hour','Date']].value_counts()

Hour  Date      
23    2022-06-06    8
7     2022-06-05    8
8     2022-01-14    8
      2022-01-13    8
      2022-01-12    8
                   ..
15    2022-06-05    8
11    2022-02-07    5
9     2022-05-11    3
19    2022-04-23    3
11    2022-05-11    1
Length: 3538, dtype: int64

In [232]:
store_date=[]
store_hour=[]

for i in tqdm(range(len(answer[['Hour','Date']].value_counts()))):
    if(answer[['Hour','Date']].value_counts().iloc[i]!=8):
        store_date.append(answer[['Hour','Date']].value_counts()[i:].rename_axis(['Hour','Date']).reset_index(name='counts')['Date'])
        store_hour.append(answer[['Hour','Date']].value_counts()[i:].rename_axis(['Hour','Date']).reset_index(name='counts')['Hour'])

100%|█████████████████████████████████████████████████████████████████████████████| 3538/3538 [00:14<00:00, 246.67it/s]


In [233]:
for i in range(len(store_date)):
    print(store_date[i].values[0])

2022-02-07
2022-05-11
2022-04-23
2022-05-11


In [234]:
for i in range(len(store_hour)):
    print(store_hour[i].values[0])

11
9
19
11


In [235]:
len(answer)

28284

In [236]:
for i in range(len(store_hour)):
    answer.drop(answer.loc[(answer['Date']==store_date[i].values[0]) & (answer['Hour']==store_hour[i].values[0])].index, inplace=True)

answer.sort_values(by=['UpdateTime'],inplace=True)
answer.reset_index(drop=True,inplace=True)
len(answer)

28272

In [253]:
answer['Power']=0.0

In [254]:
answer

,Unnamed: 0,UpdateTime,kWh,minute,Hour,Date,min_interval,Power
0,0,2022-01-04 19:07:33,549.28,7,19,2022-01-04,1,0.0
1,1,2022-01-04 19:15:54,549.58,15,19,2022-01-04,1,0.0
2,2,2022-01-04 19:15:54,549.58,15,19,2022-01-04,1,0.0
3,3,2022-01-04 19:30:23,550.08,30,19,2022-01-04,2,0.0
4,4,2022-01-04 19:30:23,550.08,30,19,2022-01-04,2,0.0
...,...,...,...,...,...,...,...,...
28267,28279,2022-06-06 23:30:22,4863.22,30,23,2022-06-06,2,0.0
28268,28280,2022-06-06 23:30:22,4863.22,30,23,2022-06-06,2,0.0
28269,28282,2022-06-06 23:45:37,4863.60,45,23,2022-06-06,3,0.0
28270,28281,2022-06-06 23:45:37,4863.60,45,23,2022-06-06,3,0.0


In [260]:
data = pd.DataFrame()
for i in tqdm(range(0,len(answer),2)):
    answer['Power'][i+1]=answer['kWh'][i+1]-answer['kWh'][i]
    data=data.append(answer[i+1:i+2],ignore_index=True)

  0%|                                                                                        | 0/14136 [00:00<?, ?it/s]C:\Users\idsl\anaconda3\envs\sister\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
100%|███████████████████████████████████████████████████████████████████████████| 14136/14136 [00:37<00:00, 376.06it/s]


In [261]:
data

,Unnamed: 0,UpdateTime,kWh,minute,Hour,Date,min_interval,Power
0,1,2022-01-04 19:15:54,549.58,15,19,2022-01-04,1,0.30
1,3,2022-01-04 19:30:23,550.08,30,19,2022-01-04,2,0.50
2,5,2022-01-04 19:45:37,550.66,45,19,2022-01-04,3,0.58
3,7,2022-01-04 19:59:23,551.14,59,19,2022-01-04,4,0.48
4,9,2022-01-04 20:15:22,551.78,15,20,2022-01-04,1,0.60
...,...,...,...,...,...,...,...,...
14131,28275,2022-06-06 22:59:46,4862.54,59,22,2022-06-06,4,0.34
14132,28277,2022-06-06 23:15:04,4862.86,15,23,2022-06-06,1,0.30
14133,28279,2022-06-06 23:30:22,4863.22,30,23,2022-06-06,2,0.36
14134,28282,2022-06-06 23:45:37,4863.60,45,23,2022-06-06,3,0.38


In [197]:
len(answer)

28272

,Unnamed: 0,UpdateTime,kWh,minute,Hour,Date,min_interval
0,0,2022-01-04 19:07:33,549.28,7,19,2022-01-04,1
1,1,2022-01-04 19:15:54,549.58,15,19,2022-01-04,1
2,2,2022-01-04 19:15:54,549.58,15,19,2022-01-04,1
3,3,2022-01-04 19:30:23,550.08,30,19,2022-01-04,2
4,4,2022-01-04 19:30:23,550.08,30,19,2022-01-04,2
...,...,...,...,...,...,...,...
28267,28279,2022-06-06 23:30:22,4863.22,30,23,2022-06-06,2
28268,28280,2022-06-06 23:30:22,4863.22,30,23,2022-06-06,2
28270,28282,2022-06-06 23:45:37,4863.60,45,23,2022-06-06,3
28269,28281,2022-06-06 23:45:37,4863.60,45,23,2022-06-06,3


In [ ]:
answer.drop(df[(df['Date']==store_date[0].values[0]].index) and )

In [147]:
answer.drop(answer['Date']==store_date[0].values[0].index)

KeyError: '[False False False ... False False False] not found in axis'

In [158]:
answer

,Unnamed: 0,UpdateTime,kWh,minute,Hour,Date,min_interval
0,0,2022-01-04 19:07:33,549.28,7,19,2022-01-04,1
1,1,2022-01-04 19:15:54,549.58,15,19,2022-01-04,1
2,2,2022-01-04 19:15:54,549.58,15,19,2022-01-04,1
3,3,2022-01-04 19:30:23,550.08,30,19,2022-01-04,2
4,4,2022-01-04 19:30:23,550.08,30,19,2022-01-04,2
...,...,...,...,...,...,...,...
28279,28279,2022-06-06 23:30:22,4863.22,30,23,2022-06-06,2
28280,28280,2022-06-06 23:30:22,4863.22,30,23,2022-06-06,2
28281,28281,2022-06-06 23:45:37,4863.60,45,23,2022-06-06,3
28282,28282,2022-06-06 23:45:37,4863.60,45,23,2022-06-06,3


In [160]:

for i in range(len(answer)):
    if():
        print(i)

5664
5665
5666
5667
5668


In [154]:
answer['Date'][i]

'2022-06-06'

'2022-02-07'

,Unnamed: 0,UpdateTime,kWh,minute,Hour,Date,min_interval
0,0,2022-01-04 19:07:33,549.28,7,19,2022-01-04,1
1,1,2022-01-04 19:15:54,549.58,15,19,2022-01-04,1
2,2,2022-01-04 19:15:54,549.58,15,19,2022-01-04,1
3,3,2022-01-04 19:30:23,550.08,30,19,2022-01-04,2
4,4,2022-01-04 19:30:23,550.08,30,19,2022-01-04,2
...,...,...,...,...,...,...,...
28279,28279,2022-06-06 23:30:22,4863.22,30,23,2022-06-06,2
28280,28280,2022-06-06 23:30:22,4863.22,30,23,2022-06-06,2
28281,28281,2022-06-06 23:45:37,4863.60,45,23,2022-06-06,3
28282,28282,2022-06-06 23:45:37,4863.60,45,23,2022-06-06,3


In [167]:
answer[(answer['Date']==store_date[0].values[0]) and (answer['Hour']==store_hour[0].values[0])].index

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [142]:
(answer['Date']==store_date[0].values[0])

0        False
1        False
2        False
3        False
4        False
         ...  
28279    False
28280    False
28281    False
28282    False
28283    False
Name: Date, Length: 28284, dtype: bool

In [ ]:
def calculate(x):
    if x <= 15:
        print(type(x))

In [ ]:
# df = pd.read_csv(f'C:/Users/idsl/用電量/提前15分鐘/Dataset/NCUE/2022-08-25/temp/力行館1F103.csv')

# df['min_interval'] = df['minute'].map(lambda x: 1 if x<=15 else (2 if x>15 and x<=30 else (3 if x>30 and x<=45 else(4 if x>45 else 0))))
# #     同區間只保留第一筆跟最後一筆
# first = df.drop_duplicates(subset=['Date','Hour','min_interval'], keep="first").reset_index(drop=True)
# last = df.drop_duplicates(subset=['Date','Hour','min_interval'], keep="last").reset_index(drop=True)
# last2 = df.drop_duplicates(subset=['Date','Hour','min_interval'], keep="last").reset_index(drop=True)
# last2.drop(last2[ last2['minute']== 59 ].index, inplace=True) #在原数据上进行删除
# # print(last2.head(10))
# answer = pd.concat([first,last,last2],axis=0)
# column= ['Date','Hour','minute']
# answer = answer.sort_values(by=column)
# # print(answer.head(10))
# #依據電表，用下一時段-前一時段
# power_next = answer.shift(-1).groupby(answer.index).first()
# power_pre = answer.groupby(answer.index).first()
# power_pre.drop(power_pre[ power_pre['minute']== 0 ].index, inplace=True) #在原数据上进行删除


# # 分為0~15、15~30、30~45、45~59
# group = answer[~ answer['TIME_TO_INTERVAL'].isin(power_pre['TIME_TO_INTERVAL'])]
# # 刪除第一筆
# group = group.drop(index=[0]).reset_index()
# # 刪除最後一筆
# group = group.drop(group.tail(1).index)
# # # 串降空的存答案
# data = pd.DataFrame()
# print(group.head(10))

# df = df.append(df2, ignore_index = True)
# for i in range(0, len(group), 2):
#     target1 = group.loc[i:i].reset_index(drop=True)
#     target2 = group.loc[i+1:i+1].reset_index(drop=True)
# #     print(target1['kWh'])
# #     print(target2['kWh'])..
#     target2['power'] = target2['kWh'].values[0] - target1['kWh'].values[0]
#     data=data.append(target2,ignore_index=True)
    
# print(data)
   
    
    





In [ ]:
print(power_next.head(6))
print(prower_pre.head(6))

In [ ]:
power_next = answer.shift(-1)
power_next

In [ ]:
temp = answer[~ answer['TIME_TO_INTERVAL'].isin(power_pre['TIME_TO_INTERVAL'])]
print(temp.head(10))

In [ ]:
answer.head(10)

In [ ]:
last2.head(10)